# COMPSCI 589 HW1

#### Name: Haochen Wang

### SECTION 0: Load Libraries

In [109]:
import sklearn.model_selection
import scipy
import numpy as np
import csv
import math
from operator import itemgetter
from collections import Counter

### SECTION 1: Evaluating KNN

In [110]:
#Load the Iris data file using python csv module

knn_file = open('iris.csv')
csvreader = csv.reader(knn_file)

knnd = []
for row in csvreader:
        knnd.append(row)

knndata = []
for row in knnd:
        c = []
        c.append(float(row[0]))
        c.append(float(row[1]))
        c.append(float(row[2]))
        c.append(float(row[3]))
        c.append(row[4])
        knndata.append(c)

# print(knndata)


In [111]:
# Implementing Helper functions

# Normalize module
def mini(col):
    min = col[0]
    for val in col:
        if val < min:
            min = val
    return min

def maxi(col):
    max = col[0]
    for val in col:
        if val > max:
            max = val
    return max

def normalizationall(col, max, min):
    newarr = []
    for val in col:
        newarr.append((val-min)/(max-min)) 
    return newarr

def normalization(col):
    min = mini(col)
    max = maxi(col)
    newarr = []
    for val in col:
        newarr.append((val-min)/(max-min)) 
    return newarr, min, max

def vote(arr):
    return max(set(arr), key=arr.count)

In [112]:
# Split the Training and Testing Data

def split(dat, ranumber):
    traknn, tesknn = sklearn.model_selection.train_test_split(dat, train_size=0.8, test_size=0.2, random_state=ranumber, shuffle=True)
    return traknn, tesknn

# trainknn, testknn = split(knndata, 589)

In [113]:
# Euclidean distance

def edistance(a, b):
    a = np.array(a)
    b = np.array(b)
    s = np.linalg.norm(a - b)
    return s

# print(edistance([1,1,1,4],[5,5,5,2]))


In [114]:
# KD-Tree

# I will do it later if I have enough time.

In [115]:
# KNN Helpers

# def seperate_d_c(data):
#     dat = []
#     cat = []
#     all = []
#     for row in data:
#         da = []
#         da.append(float(row[0]))
#         da.append(float(row[1]))
#         da.append(float(row[2]))
#         da.append(float(row[3]))
#         dat.append(da)
#         al = da.copy()
#         al.append(row[4])
#         all.append(al)
#         cat.append(row[4])
        
#     return dat, cat, all

# trainknndata, trainknncat, ktr = seperate_d_c(trainknn)
# testknndata, testknncat, kte = seperate_d_c(testknn)

def transpose(dat):
    a = []
    a.append([row[0] for row in dat])
    a.append([row[1] for row in dat])
    a.append([row[2] for row in dat])
    a.append([row[3] for row in dat])
    if len(dat[0]) > 4:
        a.append([row[4] for row in dat])
    return a

def normaltab(traindat, testdat):
    trainnom = []
    testnom = []
    i = 0
    for col in traindat:
        trarr = []
        tearr = []
        if i < 4:
            trarr, trmin, trmax = normalization(col)
            tearr = normalizationall(testdat[i], trmax, trmin)
            trainnom.append(trarr)
            testnom.append(tearr)
            i+=1
    if len(traindat) == 5:
        trainnom.append(traindat[4])
        testnom.append(testdat[4])
    return trainnom, testnom
    
def transback(dat):
    ret = []
    i = 0
    while i < len(dat[0]):
        row = []
        for col in dat:
            row.append(col[i])
        ret.append(row)
        i+=1
    return ret

def distarray(normpt, normeddat):
    pt1 = normpt[:-1]
    cat1 = normpt[-1]
    disarray = []
    for ins in normeddat:
        pt2 = ins[:-1]
        cat2 = ins[-1]
        dis = edistance(pt1, pt2)
        disarray.append([dis,cat2])
    return sorted(disarray, key=itemgetter(0))

def normflow(train, test):
    ttrainknn = transpose(train)
    ttestknn = transpose(test)
    normttrain, normttest = normaltab(ttrainknn,ttestknn)
    nrmtr, nrmte = transback(normttrain), transback(normttest)
    return nrmtr, nrmte

# we use normtr, normte.  stands for normal train & normal test.

In [116]:
#KNN

def knn(k, traindat, testdat):
    predict = []
    correct = [col[-1] for col in testdat]
    for datpt in testdat:
        distlist = distarray(datpt, traindat)
        catlist = [col[1] for col in distlist[:k]]
        predict.append(vote(catlist))

    return predict, correct

def knntrains(k, rand, dat):
    trainknn, testknn = split(dat, rand)
    normedtrain, normedtest =normflow(trainknn, testknn)
    predict, correct = knn(k, normedtrain, normedtest)
    return predict, correct

def knntraintrain(k, rand, dat):
    trainknn, testknn = split(dat, rand)
    normedtrain, normedtest =normflow(trainknn, testknn)
    predict, correct = knn(k, normedtrain, normedtrain)
    return predict, correct

def accuracy(pred, corr):
    i = 0
    blist = []
    while i < len(pred):
        blist.append(pred[i]==corr[i])
        i+=1
    return (Counter(blist)[True])/len(blist)

def kaccuracytest(k, r, data):
    p, c = knntrains(k, r, data)
    acc = accuracy(p, c)
    return acc

def kaccuracytrain(k, r, data):
    p, c = knntraintrain(k, r, data)
    acc = accuracy(p, c)
    return acc

# print(kaccuracytest(19, 589, knndata))
# print(kaccuracytrain(19, 589, knndata))

0.8666666666666667
0.9833333333333333


In [117]:
# The Statistical Process for the kNN
def statdatatest(data):
    k = 1
    result_list = []
    while k <= 51:
        random = 11589
        alist = []
        while random < 11689:
            alist.append(kaccuracytest(k, random, data))
            random += 5
        result_list.append(alist)
        k+=2
    
    return np.array(result_list)

def statdatatrain(data):
    k = 1
    result_list = []
    while k <= 51:
        random = 11589
        alist = []
        while random < 11689:
            alist.append(kaccuracytrain(k, random, data))
            random += 5
        result_list.append(alist)
        k+=2
    
    return np.array(result_list)

# narray = statdatatest(knndata)
# print(narray.std(axis=1))

[0.02808717 0.03411582 0.02708013 0.02494438 0.02768875 0.03091206
 0.02962731 0.03032234 0.02881936 0.03840573 0.04521553 0.03632416
 0.0353946  0.03411582 0.03570714 0.04381146 0.05120764 0.05423713
 0.06080479 0.06135054 0.06098725 0.05799904 0.06046119 0.06146363
 0.05857094 0.06080479]
